# MNIST Classification using PyTorch Lightning

In [1]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl
from torchmetrics.functional.classification.accuracy import accuracy

---
# PyTroch Lightning

1. model
2. optimizer 
3. data
4. training loop - The Magic
5. validation loop - The Validation Magic

In [2]:
# Defining ResNet as a Lightning Module
class ResNet(pl.LightningModule): # --> Model with residual connections
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.do = nn.Dropout(0.1)

        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2 + h1)
        logits = self.l3(do)
        return logits

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-2)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch

        # x: b * 1 * 28 * 28
        b = x.size(0)
        x = x.view(b, -1)

        # 1: Forward Prop
        logits = self(x)

        # 2: Compute Objective / Loss Function
        J = self.loss(logits, y)

        acc = accuracy(logits, y)
        self.log("acc", acc, prog_bar=True)
        return {'loss': J, "acc": acc}
    
    def validation_step(self, batch, batch_idx):
        results = self.training_step(batch, batch_idx)
        return results
    
    def validation_epoch_end(self, val_step_outputs):
        avg_val_loss = torch.tensor([x['loss'] for x in val_step_outputs]).mean()
        avg_val_acc = torch.tensor([x['acc'] for x in val_step_outputs]).mean()

        return {'val_loss': avg_val_loss, "val_acc": avg_val_acc}
    
    def train_dataloader(self):
        # Train, val split
        train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
        # train, val = random_split(train_data, [55000, 5000])
        train_loader = DataLoader(train_data, batch_size=64, num_workers=8)
        # val_loader = DataLoader(val, batch_size=32)
        return train_loader

model = ResNet()

In [3]:
trainer = pl.Trainer(max_epochs=5, gpus=1)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/indrap24/anaconda3/envs/ptDL-39/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f"you defined a {step_name} but have no {loader_name}. Skipping {stage} loop")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 50.2 K
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | do   | Dropout          | 0     
4 | loss | CrossEntropyLoss | 0     
------------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
0.220     Total estimated model params size (MB)


/home/indrap24/anaconda3/envs/ptDL-39/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Epoch 3:  41%|████      | 381/938 [00:08<00:12, 43.35it/s, loss=0.338, v_num=9, acc=0.922]

/home/indrap24/anaconda3/envs/ptDL-39/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1051: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Epoch 3:  41%|████      | 381/938 [00:21<00:32, 17.37it/s, loss=0.338, v_num=9, acc=0.922]